## Installing Libraries

In [1]:
!pip install crewai crewai_tools openai langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 650.1/650.1 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.0 MB/s 

## Import Libraries and add OpenAI key

In [2]:
import sys  # Provides access to system-specific parameters and functions
import os  # Enables interaction with the operating system
from crewai import Agent, Task, Crew, Process  # Imports core classes for defining agents, tasks, workflows, and processes
from langchain_openai import ChatOpenAI  # Integration with OpenAI's language models via LangChain
from crewai_tools import WebsiteSearchTool  # A tool for performing website searches and extracting relevant data

/usr/local/lib/python3.11/dist-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


In [4]:
from google.colab import userdata
OpenAI_API = userdata.get('OpenAI')  # Fetching the stored OpenAI API key from the Colab userdata storage
os.environ['OPENAI_API_KEY'] = OpenAI_API  # Setting the fetched API key as an environment variable for use in the application

## Initialize web search tool and Crew

In [5]:
web_tool = WebsiteSearchTool() # Initializing a WebsiteSearchTool instance for performing web searches and extracting relevant data

/usr/local/lib/python3.11/dist-packages/alembic/config.py:592: DeprecationWarning: No path_separator found in configuration; falling back to legacy splitting on spaces, commas, and colons for prepend_sys_path.  Consider adding path_separator=os to Alembic config.
  util.warn_deprecated(
/usr/local/lib/python3.11/dist-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x


In [6]:
# Initialize a ChatOpenAI instance with a specific temperature and model
llm  = ChatOpenAI(temperature=0.1, model="gpt-4o-mini")

# verbose = True, enables detailed logging

# Function to generate a blog based on a given topic
def generate_blog(topic):
    # Define the researcher agent responsible for gathering information
    researcher = Agent(
        role = "Internet Research",
        goal = f"""Conduct in-depth research on '{topic}' using reliable and diverse sources.
                 Identify key concepts, trends, examples, and statistics related to {topic}.
                 Your output should be a comprehensive, well-structured research document.""",
        verbose = True,
        llm=llm,
        backstory ="""You are a highly skilled researcher specializing in extracting, analyzing,
                     and consolidating information from the internet into actionable insights."""
    )

    # Define the content generator agent for drafting the blog
    generator = Agent(
        role='Content Generator',
        goal=f"""Draft a highly engaging and informative blog post on '{topic}'.
                 Include relevant examples, tutorials, visuals, and real-world applications.
                 Add emojis to make the blog reader-friendly. Use markdown for structure.""",
        verbose=True,
        allow_delegation=True,
        llm=llm,
        backstory="""You are an expert blogger known for creating detailed and captivating posts.
                     Your blogs are tutorial-like, enriched with real-world examples, and easy to follow."""
    )

    # Define the technical reviewer agent to ensure accuracy
    technical_reviewer = Agent(
        role="Technical Reviewer",
        goal=f"""Review the blog draft for technical accuracy, completeness, and relevance.
                 Ensure all examples, tutorials, and explanations are correct and useful.""",
        verbose=True,
        llm=llm,
        backstory="""You are a technical reviewer with expertise in validating and improving the
                     technical content of blogs, tutorials, and guides."""
    )

    # Define the copy editor agent for polishing the content
    copy_editor = Agent(
        role="Copy Editor",
        goal=f"""Polish the blog for grammar, style, and readability.
                 Ensure the content flows well and aligns with blogging best practices.""",
        verbose=True,
        llm=llm,
        backstory="""You are a skilled copy editor specializing in enhancing the quality, tone, and readability
                     of written content while preserving its original intent."""
    )

    # Define the markdown formatter agent for final formatting
    markdown_formatter = Agent(
        role="Markdown Formatter",
        goal=f"""Convert the finalized blog content into a properly structured markdown document.
                 Ensure the markdown formatting is clean, and includes proper headings, code blocks, and lists.""",
        verbose=True,
        llm=llm,
        backstory="""You are an expert in markdown formatting and structure, ensuring the content is clean,
                     organized, and ready for publishing."""
    )

    ## Define all tasks
    task_search = Task(
        description=f"""Perform extensive research on '{topic}' and compile a detailed,
                        easy-to-read research document (10,000 words).
                        Include examples, case studies, and all necessary data.""",
        expected_output=f"A well-organized research document about {topic}",
        max_inter=2,
        tools=[web_tool],
        agent=researcher
    )

    task_draft_blog = Task(
        description=f"""Use the research document to create a blog post (maximum 2,000 words).
                        Ensure the blog includes tutorials, code examples, real-world applications, and emojis.""",
        expected_output=f"A detailed and engaging blog post on {topic} in markdown format",
        agent=generator
    )

    task_review = Task(
        description=f"""Review the blog for technical accuracy and suggest any necessary improvements.
                        Ensure all examples, explanations, and tutorials are correct.""",
        expected_output=f"Feedback and revised content for technical correctness on {topic}",
        agent=technical_reviewer
    )

    task_edit = Task(
        description=f"""Edit the blog for grammar, style, and readability.
                        Ensure the content is polished and easy to read.""",
        expected_output=f"Polished and reader-friendly blog content for {topic}",
        agent=copy_editor
    )

    task_format_markdown = Task(
        description=f"""Format the finalized blog into a clean markdown .md document.
                        Ensure proper headings, lists, code blocks, and links are included.""",
        expected_output=f"A well-structured markdown file ready for publishing on {topic}",
        agent=markdown_formatter
    )

    # Define a crew to manage the agents and tasks
    crew = Crew(
        agents=[researcher, generator, technical_reviewer, copy_editor, markdown_formatter],
        tasks=[task_search, task_draft_blog, task_review, task_edit, task_format_markdown],
        verbose=True,
        process=Process.sequential)

    # Execute the process and return the result
    result = crew.kickoff()
    return result

## Running the Crew

Note: All the red, green colored text during the generation of blog is the model running the processes internally. If you want to stop seeing it, put verbose=False everywhere.

In [7]:
# Define the topic for the blog
topic = "Bagging vs Boosting in Machine Learning"
# Call the generate_blog function to create the blog on the given topic
result = generate_blog(topic)
# Print the final result of the blog generation process
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: bf7121c9-394b-441a-b3b7-0baa5a656761                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Internet Research                                                                                       │
│                                                                                                                 │
│  Task: Perform extensive research on 'Bagging vs Boosting in Machine Learning' and compile a detailed,          │
│                          easy-to-read research document (10,000 words).                                         │
│                          Include examples, case studies, and all necessary data.                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

/usr/local/lib/python3.11/dist-packages/ipywidgets/widgets/widget_output.py:111: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  if ip and hasattr(ip, 'kernel') and hasattr(ip.kernel, '_parent_header'):

/usr/local/lib/python3.11/dist-packages/ipywidgets/widgets/widget_output.py:112: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  self.msg_id = ip.kernel._parent_header['header']['msg_id']

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Markdown Formatter                                                                                      │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```markdown                                                                                                    │
│  # Bagging vs Boosting in Machine Learning                                                                      │
│                                                                                                                 │
│  ## Introduction                                                                                                │
│  In the realm of machine learning, ensemble methods are powerful techniques that combine multiple models to     │
│  improve performance. Two of the most popular ensemble methods are Bagging (Bootstrap Aggregating) and          │
│  Boosting. Both techniques aim to enhance the accuracy of models, but they do so in fundamentally different     │
│  ways. This document provides a comprehensive overview of Bagging and Boosting, including their key concepts,   │
│  differences, advantages, disadvantages, examples, and case studies.                                            │
│                                                                                                                 │
│  ## Key Concepts                                                                                                │
│                                                                                                                 │
│  ### Bagging (Bootstrap Aggregating)                                                                            │
│  - **Definition**: Bagging is an ensemble method that aims to reduce variance by training multiple models on    │
│  different subsets of the training data and averaging their predictions.                                        │
│  - **How it Works**:                                                                                            │
│    1. **Bootstrap Sampling**: Randomly sample the training dataset with replacement to create multiple          │
│  subsets.                                                                                                       │
│    2. **Model Training**: Train a separate model (often a decision tree) on each subset.                        │
│    3. **Aggregation**: Combine the predictions of all models, typically by averaging (for regression) or        │
│  voting (for classification).                                                                                   │
│  - **Goal**: To improve the stability and accuracy of machine learning algorithms by reducing overfitting.      │
│                                                                                                                 │
│  ### Boosting                                                                                                   │
│  - **Definition**: Boosting is an ensemble method that aims to reduce bias by sequentially training models,     │
│  where each new model focuses on the errors made by the previous ones.                                          │
│  - **How it Works**:                                                                                            │
│    1. **Sequential Training**: Train models one after another, with each model trying to correct the errors of  │
│  its predecessor.                                                                                               │
│    2. **Weighting**: Assign weights to instances based 

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: a580caf0-79d4-453f-b026-949b0dc21761                                                                     │
│  Agent: Markdown Formatter                                                                                      │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: bf7121c9-394b-441a-b3b7-0baa5a656761                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```markdown                                                                                      │
│  # Bagging vs Boosting in Machine Learning                                                                      │
│                                                                                                                 │
│  ## Introduction                                                                                                │
│  In the realm of machine learning, ensemble methods are powerful techniques that combine multiple models to     │
│  improve performance. Two of the most popular ensemble methods are Bagging (Bootstrap Aggregating) and          │
│  Boosting. Both techniques aim to enhance the accuracy of models, but they do so in fundamentally different     │
│  ways. This document provides a comprehensive overview of Bagging and Boosting, including their key concepts,   │
│  differences, advantages, disadvantages, examples, and case studies.                                            │
│                                                                                                                 │
│  ## Key Concepts                                                                                                │
│                                                                                                                 │
│  ### Bagging (Bootstrap Aggregating)                                                                            │
│  - **Definition**: Bagging is an ensemble method that aims to reduce variance by training multiple models on    │
│  different subsets of the training data and averaging their predictions.                                        │
│  - **How it Works**:                                                                                            │
│    1. **Bootstrap Sampling**: Randomly sample the training dataset with replacement to create multiple          │
│  subsets.                                                                                                       │
│    2. **Model Training**: Train a separate model (often a decision tree) on each subset.                        │
│    3. **Aggregation**: Combine the predictions of all models, typically by averaging (for regression) or        │
│  voting (for classification).                                                                                   │
│  - **Goal**: To improve the stability and accuracy of machine learning algorithms by reducing overfitting.      │
│                                                                                                                 │
│  ### Boosting                                                                                                   │
│  - **Definition**: Boosting is an ensemble method that aims to reduce bias by sequentially training models,     │
│  where each new model focuses on the errors made by the previous ones.                                          │
│  - **How it Works**:                                                                                            │
│    1. **Sequential Training**: Train models one after another, with each model trying to correct the errors of  │
│  its predecessor.                                     

```markdown
# Bagging vs Boosting in Machine Learning

## Introduction
In the realm of machine learning, ensemble methods are powerful techniques that combine multiple models to improve performance. Two of the most popular ensemble methods are Bagging (Bootstrap Aggregating) and Boosting. Both techniques aim to enhance the accuracy of models, but they do so in fundamentally different ways. This document provides a comprehensive overview of Bagging and Boosting, including their key concepts, differences, advantages, disadvantages, examples, and case studies.

## Key Concepts

### Bagging (Bootstrap Aggregating)
- **Definition**: Bagging is an ensemble method that aims to reduce variance by training multiple models on different subsets of the training data and averaging their predictions.
- **How it Works**:
  1. **Bootstrap Sampling**: Randomly sample the training dataset with replacement to create multiple subsets.
  2. **Model Training**: Train a separate model (often a decision tree)